# Brutus (aka Allowen)


## What
Auto-trailhead bot taker, thanks to the magic of Selenium.

## Why
My management (mid-management) keep insisting on us taking trailheads (which is a good thing), *except* that it forces us to:
* Take trailheads that are not relevant to our roles.
* They don't want to allocate time for us to take them, forcing us to either lower our already crazy utilization targets or doing overtime.
* They don't care if we do _well_. They only care that we passed the trailheads.

I also think it would make for a neat selenium full project, which I haven't gotten time to do since I did an org creator automator more than a year ago.

I'm also very curious to know how many units/points can be achieved using this silly approach.

## The code

The gist is:

1. Load dependencies
2. Open trailhead URL (modules page)
3. Login using gmail credentials (you should have a user for this already, set in the .env file)
4. Optional: get the list of all the modules, including their URLs
5. Optional: for each module, get the list of all the units, including their URLs
6. Crawl the list of units, attempting to solve them one by one. Update the json file with the results.

In [18]:
# Load Dependencies
import os
import urllib
import json
import time
from dotenv import load_dotenv, find_dotenv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

In [19]:
# Initialize variables, including webdriver
load_dotenv(find_dotenv())
driver = webdriver.Firefox(executable_path=os.environ['WEBDRIVER_PATH'])

wait = WebDriverWait(driver, 10, 1.0)
timeout = 10000

In [20]:
# Login using gmail credentials
def googleLogin(driver):
    #login button + modal
    driver.find_element(By.XPATH,"//button[@value='login']").click()
    driver.find_element(By.CLASS_NAME,'th-modal-btn__google').click()
    #login oauth page
    wait.until(EC.presence_of_element_located((By.ID,'identifierId')))
    driver.find_element(By.ID, 'identifierId').send_keys(os.environ['BRUTUS_USER'])
    driver.find_element(By.ID, 'identifierNext').click()
    wait.until(EC.element_to_be_clickable((By.ID,'passwordNext')))
    driver.find_element(By.XPATH,"//div[@id='password']//input").send_keys(os.environ['BRUTUS_PASSWORD'])
    driver.find_element(By.ID, 'passwordNext').click()


Function to create a list of trailheads

In [21]:
#get the list of all the modules, including their URLs
def crawlModules(driver):
    tTitle,tDesc,tURL,tDur,tUnits= [],[],[],[],[]
    trails=driver.find_elements_by_tag_name("article")
    
    print(str(len(trails)) + " trails found")
    
    for trail in trails:
        trail_title=trail.find_element_by_xpath(".//a").get_attribute("title")
        trail_description=trail.find_element_by_class_name("tile-description").text
        trail_url=trail.find_element_by_xpath(".//a").get_attribute("href")
        trail_duration=trail.find_element_by_class_name("progress-text").text
        trail_unitsAux=trail.find_element_by_class_name("th-button--popover-trigger").text
        units_extra=len(trail_unitsAux)-11
        trail_units=trail_unitsAux[:units_extra]

        tTitle.append(trail_title.strip())
        tDesc.append(trail_description.strip())
        tURL.append(trail_url.strip())
        tDur.append(trail_duration.strip())
        tUnits.append(trail_units.strip())

    trails_json = [{"Title":t,"Description":d,"URL":url,"Duration":dur,"Units":u,"Done":"False","UnitDetails":""} for
                  t,d,url,dur,u in zip(tTitle,tDesc,tURL,tDur,tUnits)]

    return trails_json

In [22]:
#For each unit in a module, get the titles and urls and update the trails array
def crawlUnits(driver,trails_json):
    for trail in trails_json:
        qTitle,qURL = [],[]
        driver.get(trail["URL"])

        units=driver.find_elements_by_xpath("//ul[@class='trailhead-child-items']/li")
        for unit in units:
            unit_title=unit.find_element_by_xpath(".//a//h3").text
            unit_url=unit.find_element_by_xpath(".//a").get_attribute("href")
            qTitle.append(unit_title)
            qURL.append(unit_url)

        trail["UnitDetails"]=[{"Title":t,"URL":url,"Questions":""} for
              t,url in zip(qTitle,qURL)]

    return trails_json

In [ ]:
# Crawl the list of units, attempting to solve them one by one.
def answerUnits(driver,trails_json):
    for trail in trails_json:
        # check if the "Done" value is "False"
        if (trail["Done"] == "False") or (trail["Done"] == "Skip"):
            
            units=trail["UnitDetails"]
            
            for unit in units:
                qQuestions,qOptions,qAnswers= [],[],[]
                tIndex = 0 # how many times the quiz have been done?
                tSuccess = False
                
                driver.get(unit["URL"])
                
                try:
                    unitQuiz=driver.find_element(By.CLASS_NAME,'th-quiz')
                    questions=unitQuiz.find_elements(By.CLASS_NAME,'th-quiz__question')                        
                    # print(str(len(questions)) + " questions found" )
                    
                    # initialize the questions array
                    for aux in range (0,len(questions)):
                        qQuestions.append(questions[aux].find_element(By.CLASS_NAME,'th-quiz__question-text').text)
                        qOptions.append(0)
                        qAnswers.append('')
                    
                    # try to solve the unit, one submit at a time
                    oIndex = 0
                    qDone = 0
                    while (qDone <= len(questions)):
                        # for each question, pick the answer x
                        # if it doesn't work, pick answer x+1 and so on
                        # at the end of the loop, always click submit
                        qIndex = 0
                        qDone=0
                        for question in questions:
                            question_answers=question.find_elements(By.CSS_SELECTOR,"div.th-quiz__radio_button .th-quiz__item-text")
                            question_options=question.find_elements(By.CSS_SELECTOR,"div.th-quiz__radio_button input")
                            if(len(question_options) > oIndex):
                                if(question_options[oIndex].get_attribute("disabled")):
                                    # print("Skipping. Answer found!")
                                    qDone=qDone+1
                                else:
                                    qAnswers[qIndex] = question_answers[oIndex].text
                                    qOptions[qIndex] = oIndex
                                    question_options[oIndex].click()
                            else:
                                print("couldn't select an answer for the question")
                            qIndex = qIndex+1
                        
                        #print("submit!!! " + str(oIndex) +"/q " +str(qDone) + "/i " +str(qIndex))
                        oIndex = oIndex+1
                        wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"th-button--success")))
                        submitBtn=driver.find_element(By.XPATH,"//button[@type='submit']")
                        submitBtn.click()
                        time.sleep(5)
                except NoSuchElementException:
                    try:
                        nextUnitQuiz=driver.find_element(By.XPATH,"//div[@class='th-challenge-complete__footer']//a").text
                        trail["Done"] = "Done"
                        print("Next Unit: " + nextUnitQuiz)
                    except NoSuchElementException:
                        trail["Done"] = "Skip"
                        print("Skipping this unit (Hands-On found)")
                except StaleElementReferenceException:
                    print("moving on")
                
                # update answers
                if (trail["Done"] == "False") or (trail["Done"] == "Skip"):
                    unit["Questions"]=[{"Question":q,"Option":o,"Answer":a} for
                      q,o,a in zip(qQuestions,qOptions,qAnswers)]
            # save as we go, just in case
            with open('trails.json', 'w+') as outfile:
                json.dump(trails_json, outfile)

                
    print("trails.json updated with answers info")
    return trails_json

In [ ]:
# MAIN

url = os.environ['TRAILHEAD_MODULES']
driver.get(url)
googleLogin(driver)
wait.until(EC.presence_of_element_located((By.XPATH,"//div[@data-test='header-account-name']")))

# OPTION 1: If you have never gathered the list of modules and units, use this:
#modules_json = crawlModules(driver)
#units_json = crawlUnits(driver,modules_json)

# OPTION 2: If you rather start from a json file, use this:
with open('trails.json', 'r') as readfile:
    units_json = json.load(readfile)

# call the answering bot function
trails_json=answerUnits(driver,units_json)

print ("DONE!")


Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Next Unit: Use Picklists in Formulas
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Next Unit: Enhance Your Template With App Customization Features
Next Unit: Enhance Your Template With App Customization Features
Next Unit: Enhance Your Template With App Customization Features
Next Unit: Enhance Your Template With App Customization Features
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Next Unit: Manipulate Records with DML
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Skipping this unit (Hands-On found)
Next Unit: Apply Domain Layer Principles in Apex
Skipping this unit (Hands-On found)
Next Unit: Apply Selector Layer Principles in Apex
Skipping this unit (Hands-On found)
Next Unit: Apply Service